In [2]:
# imports
%load_ext autoreload
%autoreload 2
import time
from timeit import default_timer as timer
from victor_thesis_experiments import *
from victor_thesis_utils import *
from victor_thesis_landscapes import *
from victor_thesis_plots import *
from victor_thesis_metrics import *
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import cpu_count

starting
done


In [4]:
import time
from datetime import datetime
from timeit import default_timer as timer
from qnns.cuda_qnn import UnitaryParametrization
#from victor_thesis_experiments import *
from victor_thesis_utils import *
from victor_thesis_landscapes import *
from victor_thesis_plots import *
from victor_thesis_metrics import *
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import cpu_count

# full experiment framework

#todo
def process_and_store_metrics(landscapes, conf_id):
    #calculates and stores the raw metrics, the standard deviations and medians of the metrics and have config id in name of file to match to configs.txt
    #gets 5 landscapes as input
    #conf_id
    #run_1
    #TV=...
    #...
    #run_5
    #...
    #run_avg
    #TV=...
    #...
    #run_stdv
    #TV=...
    #...
    return 0

#todo
def store_configs_to_file(unitaries, configurations):
    #write down all run configurations in a file with the run_ids
    #-
    #conf_id=16
    #data_type=orthogonal
    #num_data_points=3
    #deg_entanglement=2
    #unitary=[[a,b],[c,d]] (rough form)
    #data_batch_1=[[...]]
    #...
    #data_batch_5=[[...]]
    return 0

#todo
def generate_data_points(type_of_data, entanglement, num_data_points):
    return type_of_data*entanglement*num_data_points

# one asynchronous run will calculate 5 landscapes and their metrics
def run_single_experiment(grid_size, dimensions, data_batch, U, qnn, conf_id):
    #data batch contains 5 datapoint-sets, as we do 5 runs per unitary and then average etc.
    landscapes= []
    # for data_set in data_batch:
    #     landscapes.append(generate_loss_landscape(grid_size, dimensions, data_set, U, qnn))
    # process_and_store_metrics(landscapes, conf_id)
    now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(f"[{now}] Finished run: {conf_id}")

def run_full_experiment(num_qubits, num_unitaries = 5, num_tries = 5):
    # generate general qnn (?)
    qnn = UnitaryParametrization(num_wires=num_qubits, num_layers=1, device='cpu')
    grid_size = 2
    dimensions = 2
    unitaries = []
    #[id_unitary][id_try][type_of_data][deg_of_entanglement][num_data_points]
    configurations = []
    for _ in range(num_unitaries):
        #generate a random unitary with num_qubits qubits (why are they the same?)
        unitaries.append(torch.tensor(np.array(random_unitary_matrix(num_qubits)), dtype=torch.complex128, device="cpu"))

    
    start = time.time()
    # generate configurations (5 datapoint sets = 5 runs per config)
    conf_id = 0    
    with ProcessPoolExecutor(cpu_count()) as exe:        
        # iterate over  type of training data: 1=random, 2=orthogonal, 3=linearly dependent in H_x, 4= variable schmidt rank
        for type_of_data in range(1,5,1):   
            num_data_points_row = []
            # iterate over degree of entanglement 1 to 4
            for num_data_points in range(1,5,1):
                deg_of_entanglement_row = []
                # iterate over training data size 1 to 4
                for deg_of_entanglement in range(1,5,1):
                    # iterate over unitaries
                    unitary_row = []
                    for unitary in unitaries:       
                        data_batch_for_unitary = []
                        #iterate over number of tries/runs
                        for _ in range(1,num_tries+1,1):
                            # generate array of training data configurations [type_of_data][num_data_points][deg_of_entanglement][id_unitary][id_try]
                            data_points = generate_data_points(type_of_data, deg_of_entanglement, num_data_points)
                            data_batch_for_unitary.append(data_points)
                        # run this per configuration unitary (5 sets of data -> take average and stdv...)
                        exe.submit(run_single_experiment,grid_size, dimensions, data_batch_for_unitary, unitary, qnn, conf_id)                 
                        conf_id += 1
                        unitary_row.append(data_batch_for_unitary)                    
                    deg_of_entanglement_row.append(unitary_row)
                num_data_points_row.append(deg_of_entanglement_row)
            configurations.append(num_data_points_row)

    configurations = np.array(configurations)
    print(configurations.shape)  
    store_configs_to_file(unitaries, configurations)    
    end = time.time()
    print(f"total runtime: {np.round(end-start,2)}s")
if __name__ == '__main__':
    run_full_experiment(2)

In [ ]:
# test unitary parametrization
num_qubits = 2
qnn = UnitaryParametrization(num_wires=num_qubits, num_layers=1, device='cpu')
random_unitary = torch.tensor(np.array(random_unitary_matrix(num_qubits)), dtype=torch.complex128, device="cpu")
print("qnn",qnn.params)
print("unitary",random_unitary)
non_entangled_inputs = generate_random_datapoints(55, num_qubits, random_unitary)
non_entangled_landscape, _= generate_loss_landscape(15, 4 , non_entangled_inputs, random_unitary, qnn) 

In [ ]:
# test n-dim landscape generation
def test_ndim_landscapes():
    # hadamard U2
    qnn = get_qnn("CudaU2", list(range(1)), 1, device="cpu")
    unitary = torch.tensor(np.array([[1, 1], [1, -1]]) / np.sqrt(2), dtype=torch.complex128, device="cpu")
    ############
    # phase R
    # qnn = get_qnn("CudaR", list(range(1)), 1, device="cpu")
    # unitary = torch.tensor(np.array([[1, 0], [0, 1j]]), dtype=torch.complex128, device="cpu")
    ############
    num_ticks = 100
    data_points = 20
    # generate data points
    non_entangled_inputs = generate_random_datapoints(data_points, 1, unitary)
    # print("starting small tests")
    # # actual 3d qnn vs 2d qnn with same tick number
    # n_2d_ticks = 353
    # start = timer()
    # loss_nd, _ = generate_loss_landscape(n_2d_ticks,2, non_entangled_inputs, unitary, qnn)
    # end = timer() 
    # print(f"2d gen with* {n_2d_ticks**2} points: {np.round(end-start,3)}s")

    # qnn = get_qnn("CudaPennylane", list(range(1)), 1, device="cpu")
    # n_3d_ticks = 50
    # start = timer()
    # loss_nd, _ = generate_loss_landscape(n_3d_ticks,3, non_entangled_inputs, unitary, qnn)
    # end = timer() 
    # print(f"3d gen with {n_3d_ticks**3} points: {np.round(end-start,3)}s")

    # multi dim and tick tests
    qnn = get_qnn("CudaU2", list(range(1)), 1, device="cpu")
    print("starting full tests")
    dimensions_tests = [2, 3, 4, 5, 6, 7, 8, 9]
    num_ticks_tests = [3, 5, 7, 10, 15, 20]
    for dimensions in dimensions_tests:
        print(f"{dimensions} PARAMS")
        for ticks in num_ticks_tests:    
            start = timer()
            landscape, _= generate_loss_landscape(ticks,dimensions, non_entangled_inputs, unitary, qnn)    
            end = timer()        
            print(f"time for landscape generation with {dimensions} params and {ticks} ticks ({ticks**dimensions} points): {np.round(end-start,3)}s")
            start = timer()
            calc_scalar_curvature(landscape)  
            end = timer()        
            print(f"time for scalar curvature calculation: {np.round(end-start,3)}s")
            start = timer()
            calc_total_variation(landscape)  
            calc_IGSD(landscape)
            calc_fourier_density(landscape) 
            end = timer()        
            print(f"time for all other metrics: {np.round(end-start,3)}s")
            print("-----")
        print("##############################################################################")
        print("##############################################################################")
        print("##############################################################################")
    #test 3d stuff
test_ndim_landscapes()

In [ ]:
# n-dim tests
def n_dim_tests(
):
    qnn = get_qnn("CudaPennylane", list(range(1)), 1, device="cpu")
    unitary = torch.tensor(
        np.array([[1, 1], [1, -1]]) / np.sqrt(2), dtype=torch.complex128, device="cpu"
    )
    num_ticks = 12
    # generate data points
    non_entangled_inputs = generate_random_datapoints(5, 1, unitary)
    entangled_inputs = generate_random_datapoints(5, 2, unitary)
    z_o_inputs = get_zero_one_datapoints()
    # get 3d landscapes
    loss_z_o_3d= generate_3D_loss_landscape(
        num_ticks, z_o_inputs, unitary
    )
    loss_non_ent_3d= generate_3D_loss_landscape(
        num_ticks, non_entangled_inputs, unitary
    )
    loss_ent_3d= generate_3D_loss_landscape(
        num_ticks, entangled_inputs, unitary
    )
    landscapes = [loss_z_o_3d, loss_non_ent_3d, loss_ent_3d]
    #test total variation
    print(
        "TOTAL VARIATION: ",
        calc_total_variation(landscapes[0]),
        calc_total_variation(landscapes[1]),
        calc_total_variation(landscapes[2]),
    )
    #test igsd
    for landscape in landscapes:
        igsd = calc_IGSD(landscape)
        count = 1
        for sd in igsd:
            print(f"IGSD (dir {count}): {sd}")
            count +=1
        print("---------")
    #test fourier stuff
    print("fourier densities:")
    print(calc_fourier_density(landscapes[0]))
    print(calc_fourier_density(landscapes[1]))
    print(calc_fourier_density(landscapes[2]))

    #test scalar curvature
    print("---------")
    print("Scalar curvature absolute sums: ")
    for landscape in landscapes:
        print(np.sum(np.absolute(calc_scalar_curvature(landscape))))
    
n_dim_tests()

In [ ]:
# n-dim tests 2d
qnn = get_qnn("CudaU2", list(range(1)), 1, device="cpu")
unitary = torch.tensor(
    np.array([[1, 1], [1, -1]]) / np.sqrt(2), dtype=torch.complex128, device="cpu"
)
num_ticks = 12
# generate data points
non_entangled_inputs = generate_random_datapoints(5, 1, unitary)
entangled_inputs = generate_random_datapoints(5, 2, unitary)
z_o_inputs = get_zero_one_datapoints()
# get 3d landscapes
loss_z_o_3d= generate_2d_loss_landscape(
    num_ticks, z_o_inputs, unitary, qnn
)
loss_non_ent_3d= generate_2d_loss_landscape(
    num_ticks, non_entangled_inputs, unitary, qnn
)
loss_ent_3d= generate_2d_loss_landscape(
    num_ticks, entangled_inputs, unitary, qnn
)
landscapes = [loss_z_o_3d, loss_non_ent_3d, loss_ent_3d]
#test scalar curvature
# print(get_scalar_curvature(landscapes[0]))
# get_scalar_curvature(landscapes[1])
# get_scalar_curvature(landscapes[2])
#test total variation
print(
    "TOTAL VARIATION: ",
    calc_total_variation(landscapes[0]),
    calc_total_variation(landscapes[1]),
    calc_total_variation(landscapes[2]),
)
#test igsd
for landscape in landscapes:
    igsd = calc_IGSD(landscape)
    count = 1
    for sd in igsd:
        print(f"IGSD (dir {count}): {sd}")
        count +=1
    print("---------")
#test fourier stuff
print("fourier stuff")
fourier_landscape_z_o = get_fourier_landscape(z_o_inputs, unitary, qnn)
fourier_landscape_non_entangled = get_fourier_landscape(non_entangled_inputs, unitary, qnn)
fourier_landscape_entangled = get_fourier_landscape(entangled_inputs, unitary, qnn)
print(calc_fourier_density(landscapes[0]))
print(calc_fourier_density(landscapes[1]))
print(calc_fourier_density(landscapes[2]))
#test scalar curvature
for landscape in landscapes:
    print("SC: ",np.sum(np.absolute(calc_scalar_curvature(landscape))))

In [ ]:
# fourier testing

# hadamard U2
qnn = get_qnn("CudaU2", list(range(1)), 1, device="cpu")
unitary = torch.tensor(np.array([[1, 1], [1, -1]]) / np.sqrt(2), dtype=torch.complex128, device="cpu")
############
# phase R
# qnn = get_qnn("CudaR", list(range(1)), 1, device="cpu")
# unitary = torch.tensor(np.array([[1, 0], [0, 1j]]), dtype=torch.complex128, device="cpu")
############
num_ticks = 10
# generate data points
non_entangled_inputs = generate_random_datapoints(5, 1, unitary)
entangled_inputs = generate_random_datapoints(5, 2, unitary)
z_o_inputs = get_zero_one_datapoints()
# get 2d landscapes
loss_z_o= generate_2d_loss_landscape(
    num_ticks, z_o_inputs, unitary, qnn
)
loss_non_ent= generate_2d_loss_landscape(
    num_ticks, non_entangled_inputs, unitary, qnn
)
loss_ent= generate_2d_loss_landscape(
    num_ticks, entangled_inputs, unitary, qnn
)
landscapes = [loss_z_o, loss_non_ent, loss_ent]

#test fourier stuff
print("fourier stuff")
fourier_landscape_z_o = get_fourier_landscape(z_o_inputs, unitary, qnn,num_ticks).values
z_o_fd, z_o_f_plot = calc_fourier_density(landscapes[0])
print("FD manual custom:", z_o_fd)
fourier_landscape_non_entangled = get_fourier_landscape(non_entangled_inputs, unitary, qnn,num_ticks).values
n_e_fd, non_ent_f_plot = calc_fourier_density(landscapes[1])
print("FD manual custom:", n_e_fd)
fourier_landscape_entangled = get_fourier_landscape(entangled_inputs, unitary, qnn,num_ticks).values
e_fd, ent_f_plot = calc_fourier_density(landscapes[2])
print("FD manual custom:", e_fd)

# differences possibly explained by different loss landscape sampling
plot_row(landscapes, ["zero one", "non ent","entangled"], "Hadamard", "U2", mode="default")
print("real values")
plot_fourier_row(np.absolute(np.array([fourier_landscape_z_o, fourier_landscape_non_entangled, fourier_landscape_entangled]).real), ["zero one, with lib", "non entangled, with lib","entangled, with lib"])
print("absolute sum: ", np.sum(np.absolute(np.array([fourier_landscape_z_o, fourier_landscape_non_entangled, fourier_landscape_entangled]).real)))
plot_fourier_row(np.absolute(np.array([z_o_f_plot, non_ent_f_plot, ent_f_plot]).real), ["zero one, manual", "non entangled, manual","entangled, manual"])
print("absolute sum: ", np.sum(np.absolute(np.array([[z_o_f_plot, non_ent_f_plot, ent_f_plot]]).real)))

print("imaginary values")
plot_fourier_row(np.absolute(np.array([fourier_landscape_z_o, fourier_landscape_non_entangled, fourier_landscape_entangled]).imag), ["zero one, with lib", "non entangled, with lib","entangled, with lib"])
print("absolute sum: ", np.sum(np.absolute(np.array([fourier_landscape_z_o, fourier_landscape_non_entangled, fourier_landscape_entangled]).imag)))
plot_fourier_row(np.absolute(np.array([z_o_f_plot, non_ent_f_plot, ent_f_plot]).imag), ["zero one, manual", "non entangled, manual","entangled, manual"])
print("absolute sum: ", np.sum(np.absolute(np.array([[z_o_f_plot, non_ent_f_plot, ent_f_plot]]).imag)))

print("sum of real and imaginary values")
plot_fourier_row(np.absolute(np.array([fourier_landscape_z_o, fourier_landscape_non_entangled, fourier_landscape_entangled]).imag)+np.absolute(np.array([fourier_landscape_z_o, fourier_landscape_non_entangled, fourier_landscape_entangled]).real), ["zero one, with lib", "non entangled, with lib","entangled, with lib"])
print("absolute sum: ", np.sum(np.absolute(np.array([fourier_landscape_z_o, fourier_landscape_non_entangled, fourier_landscape_entangled]).imag)+np.absolute(np.array([fourier_landscape_z_o, fourier_landscape_non_entangled, fourier_landscape_entangled]).real)))
plot_fourier_row(np.absolute(np.array([z_o_f_plot, non_ent_f_plot, ent_f_plot]).imag)+np.absolute(np.array([z_o_f_plot, non_ent_f_plot, ent_f_plot]).real), ["zero one, manual", "non entangled, manual","entangled, manual"])
print("absolute sum: ", np.sum(np.absolute(np.array([[z_o_f_plot, non_ent_f_plot, ent_f_plot]]).imag)+np.absolute(np.array([[z_o_f_plot, non_ent_f_plot, ent_f_plot]]).real)))
# print(calc_fourier_density(landscapes[0]))
# print(calc_fourier_density(landscapes[1]))
# print(calc_fourier_density(landscapes[2]))


In [ ]:
# EXP on Hadamard
run_hadamard()

In [ ]:
# EXP on Pauli-X
run_pauli_x()

In [ ]:
# EXP on Pauli-Y
run_pauli_y()

In [ ]:
# EXP on Pauli-Z
run_pauli_z()

In [ ]:
# EXP on Phase (S)
run_phase_s()